In [1]:
#! pip install googletrans
import re
from string import punctuation
from googletrans import Translator
import pandas as pd
from pandas import DataFrame
import json

In [2]:
#Define where's the CSV file
filePath = 'C:/Users/LuisRicardoFerraz/Documents/personal/projects/book-similarity/test/tutorial/tutorial/spiders/synopsisCleaned.csv'

In [3]:
#Import CSV file to a DataFrame
df = pd.read_csv(filePath)

In [5]:
def fixDotsProblem(text):
    occurrences = re.findall('\.[A-Za-z]',text)
    for string in occurrences:
        fixing = string.replace('.','. ')
        text = text.replace(string,fixing)
    return text

In [6]:
def fixPunctuation(text):
    occurrences = re.findall('[áàãâäéèêëíìîïóòõôöúùûüA-Za-z][-./?!,":;()\'][áàãâäéèêëíìîïóòõôöúùûüA-Za-z]',text)
    for string in occurrences:
        fixing = re.sub('[-./?!,":;()\']+',' ', string)
        text = text.replace(string,fixing)
    return text

In [7]:
def removePunctuation(text):
    punctuationList = set(list(punctuation) + ['\u201c','\u201d','\u2013'])
    for signal in punctuationList:
        text = text.replace(signal,'')
    return text

In [8]:
def cleanExtraSpaces(text):
    text = re.sub('[ ]+',' ',text)
    return text

In [9]:
def cleanText(text):
    text = fixDotsProblem(text)
    text = fixPunctuation(text)
    text = removePunctuation(text)
    text = cleanExtraSpaces(text)
    text = text.lower()
    return text

In [10]:
#cleanSynopsisArray = []
#for synopsis in df['sinopse']:
#    cleanSynopsis = cleanText(str(synopsis))
#    cleanSynopsisArray.append(cleanSynopsis)
df['sinopse'] = [cleanText(str(synopsis)) for synopsis in df['sinopse']]

In [6]:
print(df.shape)

(11299, 33)


In [5]:
df = df.dropna(axis=0,how='any',subset=['sinopse'],inplace=False)

In [ ]:
translator = Translator()
df['sinopse'] = [synopsis for synopsis in df['sinopse'] if translator.detect(synopsis[:50]).lang == 'pt']
#i = 0
#for synopsis in cleanSynopsisArray:
#    i = i+1
#    if translator.detect(json.loads(synopsis)).lang == 'en':
#        print(str(synopsis))
#    if i%100 == 0:
#        print(i)

In [19]:
#Export DataFrame to a new CSV file
df.to_csv('C:/Users/LuisRicardoFerraz/Documents/personal/projects/book-similarity/test/tutorial/tutorial/spiders/.csv')